---
# 
---

In this notebook, 

1. 


### Importing necessary libraries

In [1]:
import os
import glob

import pandas as pd
import numpy as np

from geopy import distance

from id_station import stn_id 

### Define files

In [2]:
stn_inventory = '../data/ECCC/Station_Inventory_EN_AdminRegion.csv'

In [3]:
admin_region = ['Montréal']

data_dir = '../data/ECCC/daily'

filename = 'climate_daily_QC_'

frequency = 'DLY'

out_dir = '../data/ECCC/processed/daily'

outfile = os.path.join(out_dir, 'daily_processed.csv')

### Define variables

**Daily**

In [4]:
Tmax = 'Max Temp (°C)'
Tmin =  'Min Temp (°C)' 
Tmean = 'Mean Temp (°C)'

cols = ['Max Temp (°C)', 'Max Temp Flag', 'Min Temp (°C)', 'Min Temp Flag',
       'Mean Temp (°C)', 'Mean Temp Flag', 'Heat Deg Days (°C)',
       'Heat Deg Days Flag', 'Cool Deg Days (°C)', 'Cool Deg Days Flag',
       'Total Rain (mm)', 'Total Rain Flag', 'Total Snow (cm)',
       'Total Snow Flag', 'Total Precip (mm)', 'Total Precip Flag',
       'Snow on Grnd (cm)', 'Snow on Grnd Flag', 'Dir of Max Gust (10s deg)',
       'Dir of Max Gust Flag', 'Spd of Max Gust (km/h)',
       'Spd of Max Gust Flag']

**Monthly**

### Read Station Inventory file

In [5]:
stns = pd.read_csv(stn_inventory)
stns.head()

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,...,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry,admin
0,BARRIERE STONEHAM,QUEBEC,7010478,5205,NaN,NaN,47.17,-71.25,471000000,-711500000,...,1963,1976,NaN,NaN,1963.0,1976.0,1963.0,1976.0,POINT (-71.25 47.17),Capitale-Nationale
1,BARRIERE TOURILLI,QUEBEC,7010480,5206,NaN,NaN,47.17,-71.62,471000000,-713700000,...,1949,1960,NaN,NaN,1949.0,1960.0,1949.0,1960.0,POINT (-71.62 47.17),Capitale-Nationale
2,BEAUPORT,QUEBEC,7010565,27803,71578.0,XBO,46.84,-71.20,465013000,-711150000,...,1999,2025,1999.0,2024.0,1999.0,2025.0,NaN,NaN,POINT (-71.2 46.84),Capitale-Nationale
3,BEAUPORT,QUEBEC,7010566,5207,NaN,NaN,46.88,-71.20,465300000,-711200000,...,1982,1985,NaN,NaN,1982.0,1985.0,1982.0,1985.0,POINT (-71.2 46.88),Capitale-Nationale
4,BERTHIERVILLE,QUEBEC,7010720,5208,NaN,NaN,46.05,-73.18,460300000,-731100000,...,1919,1995,NaN,NaN,1919.0,1995.0,1919.0,1995.0,POINT (-73.18 46.05),Lanaudière


### Keep only stations within the desired administrative region

In [6]:
stns = stns[stns['admin'].isin(admin_region)]
stns

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,...,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry,admin
160,COTE ST LUC,QUEBEC,7021945,5343,NaN,NaN,45.45,-73.67,452700000,-734000000,...,1960,1963,NaN,NaN,1960.0,1963.0,1960.0,1963.0,POINT (-73.67 45.45),Montréal
217,LA SALLE,QUEBEC,7024118,5391,NaN,NaN,45.43,-73.62,452600000,-733700000,...,1973,1983,NaN,NaN,1973.0,1983.0,1973.0,1983.0,POINT (-73.62 45.43),Montréal
220,LAVAL DES RAPIDES,QUEBEC,7024256,5394,NaN,NaN,45.53,-73.70,453200000,-734200000,...,1965,1976,NaN,NaN,1965.0,1976.0,1965.0,1976.0,POINT (-73.7 45.53),Montréal
226,MACDONALD COLLEGE,QUEBEC,7024400,5400,NaN,NaN,45.42,-73.93,452500000,-735600000,...,1906,1976,NaN,NaN,1906.0,1976.0,1906.0,1976.0,POINT (-73.93 45.42),Montréal
234,MCTAVISH,QUEBEC,7024745,10761,71612.0,WTA,45.50,-73.58,453017070,-733445000,...,1994,2025,1994.0,2024.0,1994.0,2025.0,1994.0,2025.0,POINT (-73.58 45.5),Montréal
239,MOBILE UPPER AIR STATION-QUEBEC,QUEBEC,7025000,47888,71054.0,EUQ,45.43,-73.93,452536000,-735544040,...,2014,2021,2014.0,2021.0,2017.0,2021.0,NaN,NaN,POINT (-73.93 45.43),Montréal
242,MONTREAL ADAC A,QUEBEC,7025228,8343,NaN,NaN,45.48,-73.55,452900000,-733300000,...,1974,1976,1974.0,1976.0,NaN,NaN,NaN,NaN,POINT (-73.55 45.48),Montréal
245,MONTREAL/PIERRE ELLIOTT TRUDEAU INTL A,QUEBEC,7025250,5415,71627.0,YUL,45.47,-73.75,452800000,-734500000,...,1941,2013,1953.0,2013.0,1941.0,2013.0,1941.0,2013.0,POINT (-73.75 45.47),Montréal
246,MONTREAL INTL A,QUEBEC,7025251,51157,71627.0,YUL,45.47,-73.74,452814000,-734427000,...,2013,2025,2013.0,2024.0,2013.0,2025.0,NaN,NaN,POINT (-73.74 45.47),Montréal
247,MONTREAL-EST,QUEBEC,7025252,26855,NaN,WPQ,45.63,-73.55,453805000,-733311000,...,1994,2008,1994.0,2008.0,1994.0,2008.0,NaN,NaN,POINT (-73.55 45.63),Montréal


### Creating a dictionary of the reference station to preprocess

In [7]:
tmp = []

for key in stn_id.keys():
    for id in stn_id[key]:

        lat = stns[stns['Climate ID'] == str(id)]['Latitude (Decimal Degrees)'].values[0]
        lon = stns[stns['Climate ID'] == str(id)]['Longitude (Decimal Degrees)'].values[0]

        admin = stns[stns['Climate ID'] == str(id)]['admin'].values[0]

        tmp.append({'Station': key, 
                    'Climate ID': id, 
                    'Latitude': lat, 
                    'Longitude': lon, 
                    'admin': admin})
        
ref_stn = pd.DataFrame(tmp)
ref_stn

,Station,Climate ID,Latitude,Longitude,admin
0,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal
1,Montreal (McTavish/McGill),7024745,45.5,-73.58,Montréal


### Compute the distance from the stations to the reference stations

In [8]:

dist_stn_ref = []

for station, group in ref_stn.groupby(['Station']):

    tmp_stns = stns[stns['admin'].isin(group['admin'])]
    tmp_ref_stn = ref_stn[ref_stn['admin'].isin(group['admin'])]

    for index, ref in tmp_ref_stn.iterrows():
        for idx, stn in tmp_stns.iterrows() :

            if stn['Climate ID'] not in group['Climate ID'].values :
                ref_lat_lon = (ref['Latitude'], ref['Longitude'])
                stn_lat_lon = (stn['Latitude (Decimal Degrees)'], stn['Longitude (Decimal Degrees)'])
            
                dist = distance.distance(ref_lat_lon, stn_lat_lon).km

                dist_stn_ref.append({'Station' : ref['Station'],
                                     'Climate ID' : ref['Climate ID'],
                                     'Latitude': ref['Latitude'],
                                     'Longitude':  ref['Longitude'],
                                     'admin': admin,
                                     'Nearby Climate ID': stn['Climate ID'],
                                     'Distance (km)': dist
            })
            
            
dist_stn_ref = pd.DataFrame(dist_stn_ref)

dist_stn_ref

,Station,Climate ID,Latitude,Longitude,admin,Nearby Climate ID,Distance (km)
0,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7021945,8.966881
1,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024118,8.385239
2,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024256,9.951661
3,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024400,28.782395
4,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025000,28.456381
5,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025228,3.231210
6,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025250,13.702253
7,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025251,12.945362
8,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025252,14.637152
9,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025257,8.125190


### Reading the data files of reference stations

In [9]:
df = pd.DataFrame()

for station, group in ref_stn.groupby(['Station']):

    for idx, ref in group.iterrows():
        start = int(stns[stns['Climate ID'] == ref['Climate ID']][f'{frequency} First Year'].values[0])
        end = int(stns[stns['Climate ID'] == ref['Climate ID']][f'{frequency} Last Year'].values[0])

        print(start, end)

        for year in list(range(start,end+1)):
            
            path = os.path.join(data_dir, f'*{ref['Climate ID']}_{year}*.csv')
            file = glob.glob(path)
            #print(file[0])
            data = pd.read_csv(file[0], encoding="ISO-8859-1")

            data.insert(0,'Station', ref['Station'])

            df = pd.concat([df, data])       
            
if frequency == 'DLY' :
    df['Date/Time'] = pd.to_datetime(df['Date/Time'])
    df.set_index('Date/Time', inplace=True)

if frequency == 'MLY' :
    #df['Date/Time'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1)) 
    df['Date/Time'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1)) + pd.offsets.MonthEnd(0)
    df.set_index('Date/Time', inplace=True)

    full_range = set(pd.date_range(start=df.index.min(), end =  df.index.max(), freq="ME"))
    full_range

    missing_dates = full_range.difference(set(df.index))

    missing_df = pd.DataFrame(index=list(missing_dates))

    missing_df['Climate ID'] = None 
    missing_df['Station'] = None 
    missing_df['Year'] = None
    missing_df['Month'] = None

    df = pd.concat([df, missing_df])

    df = df.sort_index()

    df['Station'] = df['Station'].ffill()
    df['Climate ID'] = df['Climate ID'].ffill()
    
    #df = df.asfreq('M')
    #df.index.freq = 'ME'
    #df = df.resample('ME').asfreq() 
    #new_range = pd.date_range(start=df['Date/Time'].min(), end =  df['Date/Time'].max(), freq="ME")
    
    #print(new_range)
    #df = df.set_index('Date/Time').reindex(new_range)
    #df = df.reindex(new_range)
    #df = df.asfreq('ME')
    
#df.set_index('Date/Time', inplace=True)

df['Climate ID'] = df['Climate ID'].ffill().astype(int).astype(str)

#df[(df['Year'] > 1991) & (df['Year'] < 1996)].asfreq('M')
df

1871 1993
1994 2025


,Station,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
Date/Time,,,,,,,,,,,,,,,,,,,,,
1871-01-01,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1871,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-01-02,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1871,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-01-03,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1871,1,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-01-04,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1871,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-01-05,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1871,1,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,12,27,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-28,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,12,28,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-29,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,12,29,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reading the data files of nearby stations

In [10]:
near_df = pd.DataFrame()

for station, group in dist_stn_ref.groupby(['Station']):

    for near in group['Nearby Climate ID'].unique():
        if  ~np.isnan(stns[stns['Climate ID'] == near][f'{frequency} First Year'].values[0]) :
            start = int(stns[stns['Climate ID'] == near][f'{frequency} First Year'].values[0])
            end = int(stns[stns['Climate ID'] == near][f'{frequency} Last Year'].values[0])

            print(start, end)

            for year in list(range(start,end+1)):
                path = os.path.join(data_dir, f'*{near}_{year}*.csv')
                file = glob.glob(path)
                #print(file[0])
                data = pd.read_csv(file[0], encoding="ISO-8859-1")

                near_df = pd.concat([near_df, data])      


if frequency == 'DLY' :
    near_df['Date/Time'] = pd.to_datetime(near_df['Date/Time'])

if frequency == 'MLY' :
    near_df['Date/Time'] = pd.to_datetime(near_df[['Year', 'Month']].assign(day=1)) + pd.offsets.MonthEnd(0)
    

near_df = near_df.dropna(subset=cols, how='all')

near_df = near_df.dropna(subset=[Tmax, Tmin, Tmean], how='any')

near_df['Climate ID'] = near_df['Climate ID'].astype(str)

near_df            

1960 1963
1973 1983
1965 1976
1906 1976
2017 2021
1941 2013
2013 2025
1994 2008
1948 1989
1956 1985
1971 1992
1964 1964
1948 1950
1973 1985
1973 1982
1973 2024
1969 1992
1952 2023
1931 1967
1993 2025
2002 2025


,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
255,-73.67,45.45,COTE ST LUC,7021945,1960-09-12,1960,9,12,NaN,15.6,...,0.0,NaN,22.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,-73.67,45.45,COTE ST LUC,7021945,1960-09-13,1960,9,13,NaN,18.3,...,0.0,NaN,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,-73.67,45.45,COTE ST LUC,7021945,1960-09-14,1960,9,14,NaN,17.2,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,-73.67,45.45,COTE ST LUC,7021945,1960-09-15,1960,9,15,NaN,18.3,...,0.0,NaN,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,-73.67,45.45,COTE ST LUC,7021945,1960-09-16,1960,9,16,NaN,16.7,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,-73.74,45.47,MONTREAL/PIERRE ELLIOTT TRUDEAU INTL,702S006,2025-03-24,2025,3,24,NaN,3.7,...,NaN,NaN,5.7,NaN,0.0,NaN,12.0,NaN,36.0,NaN
83,-73.74,45.47,MONTREAL/PIERRE ELLIOTT TRUDEAU INTL,702S006,2025-03-25,2025,3,25,NaN,4.4,...,NaN,NaN,0.5,NaN,0.0,NaN,26.0,NaN,38.0,NaN
84,-73.74,45.47,MONTREAL/PIERRE ELLIOTT TRUDEAU INTL,702S006,2025-03-26,2025,3,26,NaN,4.7,...,NaN,NaN,0.2,NaN,0.0,NaN,25.0,NaN,38.0,NaN
85,-73.74,45.47,MONTREAL/PIERRE ELLIOTT TRUDEAU INTL,702S006,2025-03-27,2025,3,27,NaN,7.7,...,NaN,NaN,0.8,NaN,0.0,NaN,25.0,NaN,48.0,NaN


### Looking for missing values

In [11]:
df.isna().sum()

Station                          0
Longitude (x)                    0
Latitude (y)                     0
Station Name                     0
Climate ID                       0
Year                             0
Month                            0
Day                              0
Data Quality                 56613
Max Temp (°C)                 1969
Max Temp Flag                56299
Min Temp (°C)                 1935
Min Temp Flag                56223
Mean Temp (°C)                1987
Mean Temp Flag               56287
Heat Deg Days (°C)            1987
Heat Deg Days Flag           56287
Cool Deg Days (°C)            1987
Cool Deg Days Flag           56287
Total Rain (mm)              12373
Total Rain Flag              45112
Total Snow (cm)              12384
Total Snow Flag              45519
Total Precip (mm)             2311
Total Precip Flag            50795
Snow on Grnd (cm)            41653
Snow on Grnd Flag            55487
Dir of Max Gust (10s deg)    53781
Dir of Max Gust Flag

### Finding min and max Date/Time of available data

In [12]:
tmp_avail = df.dropna(subset=cols, how='all')

start = tmp_avail.index.min()
end = tmp_avail.index.max()
end = '2025-03-28'

### Replacing missing values for the temperatures with data from the nearest available stations

In [13]:

for name, group in df.groupby(['Station Name'], dropna=False):
    
    clim_id = group['Climate ID'].unique()

    start_stn = group.index.min()
    end_stn = group.index.max()

    tmp_period = df[(df.index >= start_stn) & (df.index <= end_stn)]

    tmp_period = tmp_period[(tmp_period[Tmax].isna()) | \
                            (tmp_period[Tmin].isna()) | \
                            (tmp_period[Tmean].isna())]
    
    for idx, row in tmp_period.iterrows() :

        tmp = near_df[near_df['Date/Time'] == pd.to_datetime(str(idx))]

        if len(tmp) == 0 :
            start = idx
        else :
            print(idx)
            near_idx = dist_stn_ref[(dist_stn_ref['Climate ID'].isin(clim_id)) & (dist_stn_ref['Nearby Climate ID'].isin(tmp['Climate ID'].values.astype(str)))]['Distance (km)'].idxmin()

            nearest = dist_stn_ref.loc[near_idx]['Nearby Climate ID']

            new_row = tmp[tmp['Climate ID'] == nearest]
            
            new_row.insert(0, 'Station', group['Station'].unique())
            print('new', new_row)

            df.loc[pd.to_datetime(idx)] = new_row.iloc[0]


1994-01-01 00:00:00
new                       Station  Longitude (x)  Latitude (y)  \
0  Montreal (McTavish/McGill)         -73.75         45.47   

                             Station Name Climate ID  Date/Time  Year  Month  \
0  MONTREAL/PIERRE ELLIOTT TRUDEAU INTL A    7025250 1994-01-01  1994      1   

   Day  Data Quality  ...  Total Snow (cm) Total Snow Flag  Total Precip (mm)  \
0    1           NaN  ...              3.8             NaN                3.4   

  Total Precip Flag  Snow on Grnd (cm) Snow on Grnd Flag  \
0               NaN               14.0               NaN   

   Dir of Max Gust (10s deg) Dir of Max Gust Flag  Spd of Max Gust (km/h)  \
0                       22.0                  NaN                      44   

  Spd of Max Gust Flag  
0                  NaN  

[1 rows x 32 columns]
1994-01-02 00:00:00
new                       Station  Longitude (x)  Latitude (y)  \
1  Montreal (McTavish/McGill)         -73.75         45.47   

                             

### Making sure that the temperature no longer has missing values

In [14]:
start

Timestamp('1891-10-21 00:00:00')

In [15]:
df[(df.index > start) & (df.index < end)].isna().sum()

Station                          0
Longitude (x)                    0
Latitude (y)                     0
Station Name                     0
Climate ID                       0
Year                             0
Month                            0
Day                              0
Data Quality                 48735
Max Temp (°C)                    0
Max Temp Flag                48650
Min Temp (°C)                    0
Min Temp Flag                48569
Mean Temp (°C)                   0
Mean Temp Flag               48642
Heat Deg Days (°C)               0
Heat Deg Days Flag           48642
Cool Deg Days (°C)               0
Cool Deg Days Flag           48642
Total Rain (mm)              10549
Total Rain Flag              37720
Total Snow (cm)              10557
Total Snow Flag              38046
Total Precip (mm)              482
Total Precip Flag            43631
Snow on Grnd (cm)            32530
Snow on Grnd Flag            47605
Dir of Max Gust (10s deg)    45537
Dir of Max Gust Flag

### Writing proceessed data to file

In [16]:
df[(df.index > start) & (df.index < end)].to_csv(outfile)